In [1]:
#Preapre training data

import json

train_path = 'C:\\Users\\maria\\Desktop\\nlp\\Assignment 2\\project-files\\project-files\\train.json'
dev_path = 'C:\\Users\\maria\\Desktop\\nlp\\Assignment 2\\project-files\\project-files\\dev.json'
neg_path = 'C:\\Users\\maria\\Desktop\\nlp\\Assignment 2\\project-files\\project-files\\neg-train-output.json'

encode_path = ('train.json','dev.json')


def get_content_from_json(file_path):
    document_list = []
    label_list = []
    with open(file_path) as json_file:
        if file_path.endswith(encode_path):
            data = json.load(json_file)   
            for item in data:
                text = data[item]['text'].encode('ascii','ignore').decode("utf-8")  #Encode Decode \u
                document_list.append(text) 
                label_list.append(data[item]['label'])
        else:  
            data = json.load(json_file)   
            for item in data:
                text = data[item]['text']
                document_list.append(text) 
                label_list.append(data[item]['label'])
            
    return document_list, label_list

pos_data,pos_label= get_content_from_json(train_path)
neg_data,neg_label = get_content_from_json(neg_path)
dev_data,dev_label = get_content_from_json(dev_path)



In [2]:
train_data = pos_data+neg_data
train_label = pos_label+neg_label

In [3]:
#Prepare test data
import json


test_path = 'C:\\Users\\maria\\Desktop\\nlp\\Assignment 2\\project-files\\project-files\\test-unlabelled.json'


def get_content_from_json(file_path):
    document_list = []
    with open(file_path) as json_file:
        data = json.load(json_file)   
        for item in data:
            text = data[item]['text'].encode('ascii','ignore').decode("utf-8")  #Encode Decode \u
            document_list.append(text)                
    return document_list


test_data = get_content_from_json(test_path)

len(test_data)

1410

In [4]:
#Preprocessing

import string
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from collections import Counter
from nltk.stem.porter import PorterStemmer
from nltk.corpus import wordnet

porter_stemmer = PorterStemmer()
stopwords = set(stopwords.words('English'))
table = str.maketrans('', '', string.punctuation)

def document_preprocessing(data):
    
    token_clean_corpus= []
    #tokenize
    token_corpus =  [word_tokenize(doc) for doc in data]

    for doc in token_corpus:
        #remove punctuation
        strip_token = [word.translate(table) for word in doc]
        #handle casing
        lower_token = [word.lower() for word in strip_token]
        #remove numbers 
        only_words = [word for word in lower_token if word.isalpha()]
        #remove stopwords
        token_no_stopword = [word for word in only_words if word not in stopwords]
        #stem words
        stem_token = [porter_stemmer.stem(word) for word in token_no_stopword]
        token_clean_corpus.append(stem_token)
        
    return token_clean_corpus
    
train_preprocessed_data = document_preprocessing(train_data)
dev_preprocessed_data = document_preprocessing(dev_data) 
test_preprocessed_data = document_preprocessing(test_data) 
len(train_preprocessed_data),len(dev_preprocessed_data),len(test_preprocessed_data)


(2804, 100, 1410)

In [5]:

def join_text(preprocess_data):
    join_data = [' '.join(i) for i in preprocess_data]
    return join_data


train_sent =  join_text(train_preprocessed_data)
dev_sent = join_text(dev_preprocessed_data)
test_sent = join_text(test_preprocessed_data)


In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer

tf_idf = TfidfVectorizer()
train_set = tf_idf.fit_transform(train_sent)
develop_set = tf_idf.transform(dev_sent)
test_set = tf_idf.transform(test_sent)

In [7]:
from sklearn.svm import SVC
from sklearn.metrics import f1_score,confusion_matrix,classification_report


svc = SVC(C=10)
svc.fit(train_set,train_label)
pred=svc.predict(develop_set)
score = f1_score(dev_label, pred)
print('F1 Score: %.3f' % score)
cm = confusion_matrix(dev_label,pred)
print(cm)
print(classification_report(dev_label,pred))

F1 Score: 0.898
[[46  4]
 [ 6 44]]
              precision    recall  f1-score   support

           0       0.88      0.92      0.90        50
           1       0.92      0.88      0.90        50

    accuracy                           0.90       100
   macro avg       0.90      0.90      0.90       100
weighted avg       0.90      0.90      0.90       100



### **Train model on train and dev data**

In [8]:
posdev_data = [dev_data[idx] for idx,val in enumerate(dev_label) if val == 1]
posdev_label = [1] * len(posdev_data)
negdev_data = [dev_data[idx] for idx,val in enumerate(dev_label) if val == 0]
negdev_label = [0] * len(negdev_data)

In [9]:
pos_data = pos_data+posdev_data
pos_label = pos_label+posdev_label

In [10]:
neg_data= neg_data+negdev_data
neg_label = neg_label+negdev_label

In [11]:
train_data = pos_data + neg_data
train_label = pos_label + neg_label

In [12]:
train_preprocessed_data = document_preprocessing(train_data)
dev_preprocessed_data = document_preprocessing(dev_data) 
test_preprocessed_data = document_preprocessing(test_data)

In [13]:
train_sent =  join_text(train_preprocessed_data)
dev_sent = join_text(dev_preprocessed_data)
test_sent = join_text(test_preprocessed_data)

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer

tf_idf = TfidfVectorizer()
train_set = tf_idf.fit_transform(train_sent)
develop_set = tf_idf.transform(dev_sent)
test_set = tf_idf.transform(test_sent)

In [15]:
from sklearn.svm import SVC
from sklearn.metrics import f1_score,confusion_matrix,classification_report


svc = SVC(C=10)
svc.fit(train_set,train_label)
pred=svc.predict(develop_set)
score = f1_score(dev_label, pred)
print('F1 Score: %.3f' % score)
cm = confusion_matrix(dev_label,pred)
print(cm)
print(classification_report(dev_label,pred))

F1 Score: 1.000
[[50  0]
 [ 0 50]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        50
           1       1.00      1.00      1.00        50

    accuracy                           1.00       100
   macro avg       1.00      1.00      1.00       100
weighted avg       1.00      1.00      1.00       100



In [16]:
##predict test and convert to json file

test_pred = svc.predict(test_set)

pred_dict = {}

for i, val in enumerate(test_pred):
    pred_dict["test-"+str(i)] = {"label": int(val)}

with open('C:\\Users\\maria\\Desktop\\nlp\\Assignment 2\\project-files\\project-files\\test-output.json', 'w') as outfile:
    json.dump(pred_dict, outfile)